In [26]:
import pandas as pd

# Cargar CSV separado por comas
df = pd.read_csv("AVN-6500-01_Shelf Inventory_2025-06-20 23-40-22.csv")
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
# Mostrar contenido
print(df.head())


   Shelf   Slot      Slot Type  \
0      1    \t-  \tBack Plane    
1      1    \t1         \tCard   
2      1  \t1-2         \tQSFP   
3      1  \t1-3         \tQSFP   
4      1    \t2         \tCard   

                                           Card Type    Physical PEC Release  \
0        \t6500 2-Slot Optical Type 2 Shelf Assembly      \tNTK503LA  \t 05    
1                               \tWLAi MOTR 4xQSFP28      \tNTK538DR  \t 05    
2  \t100GBASE-LR4, 4x25G, WDM, SMF, 1310nm, 10km ...  \t160-9401-900   \t006   
3  \t100GBASE-LR4, 4x25G, WDM, SMF, 1310nm, 10km ...  \t160-9401-900   \t006   
4                                      \tMLA3 C-Band      \tNTK552GA  \t 11    

  Card Width          CLEI         Serial #  Mfg. Date             Age  \
0        \t-  \tWOML100ERA   \tNNTMRT1CC1FL  \t2023-13             \t-   
1        \t-  \tWOWUBE8EAA   \tNNTMRT19V183  \t2023-09  \t00-359-13-18   
2        \t-  \tWOTRD2ZFAA  \tFNSRMYU6CA1KC        \t-             \t-   
3        \t-  \tWO

In [27]:
df.drop(columns = ['Card Width','Mfg. Date','Age','On Since','SNMP Index','Current Temperature','Average Temperature','Additional Information'], inplace=True)
df.dropna(axis=1, how='all')
df = df.replace('\t', '', regex=True)
print(df.head())

   Shelf Slot    Slot Type                                          Card Type  \
0      1    -  Back Plane           6500 2-Slot Optical Type 2 Shelf Assembly   
1      1    1         Card                                 WLAi MOTR 4xQSFP28   
2      1  1-2         QSFP  100GBASE-LR4, 4x25G, WDM, SMF, 1310nm, 10km QS...   
3      1  1-3         QSFP  100GBASE-LR4, 4x25G, WDM, SMF, 1310nm, 10km QS...   
4      1    2         Card                                        MLA3 C-Band   

   Physical PEC Release        CLEI       Serial #  
0      NTK503LA     05   WOML100ERA   NNTMRT1CC1FL  
1      NTK538DR     05   WOWUBE8EAA   NNTMRT19V183  
2  160-9401-900     006  WOTRD2ZFAA  FNSRMYU6CA1KC  
3  160-9401-900     006  WOTRD2ZFAA  FNSRMYU6CA1HU  
4      NTK552GA     11   WOGUAS9VAA   NNTMRT19CEE7  


In [28]:
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
from openpyxl.utils import get_column_letter

# Crear workbook y hoja
wb = openpyxl.Workbook()
ws = wb.active
ws.title = "Datos procesados"

# Definición de formatos
header_fill = PatternFill("solid", fgColor="969696")  # gris (150,150,150)
header_font = Font(bold=True, color="FFFFFF")
center_alignment = Alignment(horizontal='center', vertical='center')

# Bordes finos
thin_border = Border(
    left=Side(style='thin'),
    right=Side(style='thin'),
    top=Side(style='thin'),
    bottom=Side(style='thin')
)

# Insertar el DataFrame al Excel
for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True), start=1):
    ws.append(row)
    
    for c_idx, value in enumerate(row, start=1):
        cell = ws.cell(row=r_idx, column=c_idx)
        cell.alignment = center_alignment
        cell.border = thin_border

        # Si es la primera fila (encabezado), aplicar formato especial
        if r_idx == 1:
            cell.font = header_font
            cell.fill = header_fill

# Ajustar anchos de columnas
for i, col in enumerate(df.columns, start=1):
    max_length = max((len(str(cell.value)) for cell in ws[get_column_letter(i)]), default=0)
    
    if col in ['Shelf', 'Slot']:
        ws.column_dimensions[get_column_letter(i)].width = 5
    elif col in ['Release']:
        ws.column_dimensions[get_column_letter(i)].width = 8
    else:
        ws.column_dimensions[get_column_letter(i)].width = max(10, max_length + 3)

# Congelar primera fila (opcional, recomendado)
ws.freeze_panes = "A2"

# Guardar archivo
wb.save("archivo_formateado.xlsx")


In [ ]:
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
from openpyxl.utils import get_column_letter
import pandas as pd

# ----------------------------
# CONFIGURACIÓN DE FORMATOS
# ----------------------------
header_fill = PatternFill("solid", fgColor="969696")
header_font = Font(bold=True, color="FFFFFF")
center_alignment = Alignment(horizontal='center', vertical='center')

thin_border = Border(
    left=Side(style='thin'),
    right=Side(style='thin'),
    top=Side(style='thin'),
    bottom=Side(style='thin')
)

# ----------------------------
# CARGA Y LIMPIEZA DEL CSV
# ----------------------------
df = pd.read_csv("AVN-6500-01_Shelf Inventory_2025-06-20 23-40-22.csv")
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.drop(columns = ['Card Width','Mfg. Date','Age','On Since','SNMP Index','Current Temperature','Average Temperature','Additional Information'], inplace=True)
df = df.replace('\t', '', regex=True)

# ----------------------------
# CREAR WORKBOOK
# ----------------------------
wb = openpyxl.Workbook()
wb.remove(wb.active)  # Quitar hoja vacía por defecto

# ----------------------------
# CREAR UNA HOJA POR CADA Shelf
# ----------------------------
for shelf_num, group_df in df.groupby('Shelf'):
    ws = wb.create_sheet(title=f"Shelf_{shelf_num}")

    # Insertar encabezado + datos
    for r_idx, row in enumerate(dataframe_to_rows(group_df, index=False, header=True), start=1):
        ws.append(row)

        for c_idx, value in enumerate(row, start=1):
            cell = ws.cell(row=r_idx, column=c_idx)
            cell.alignment = center_alignment
            cell.border = thin_border

            # Encabezado (primera fila)
            if r_idx == 1:
                cell.font = header_font
                cell.fill = header_fill

    # Ajustar anchos de columnas
    for i, col in enumerate(group_df.columns, start=1):
        max_length = max((len(str(cell.value)) for cell in ws[get_column_letter(i)]), default=0)
        if col in ['Shelf', 'Slot']:
            ws.column_dimensions[get_column_letter(i)].width = 5
        elif col in ['Release']:
            ws.column_dimensions[get_column_letter(i)].width = 8
        else:
            ws.column_dimensions[get_column_letter(i)].width = max(10, max_length + 3)

    # Congelar encabezado
    ws.freeze_panes = "A2"
    ws.auto_filter.ref = ws.dimensions  # Agregar filtro automático

# ----------------------------
# GUARDAR ARCHIVO
# ----------------------------
wb.save("archivo_por_shelf.xlsx")


In [33]:
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
from openpyxl.utils import get_column_letter
import pandas as pd

# ----------------------------
# CONFIGURACIÓN DE FORMATOS
# ----------------------------
header_fill = PatternFill("solid", fgColor="969696")  # Gris encabezado
header_font = Font(bold=True, color="FFFFFF")
center_alignment = Alignment(horizontal='center', vertical='center')

thin_border = Border(
    left=Side(style='thin'),
    right=Side(style='thin'),
    top=Side(style='thin'),
    bottom=Side(style='thin')
)

# ----------------------------
# FUNCIONES AUXILIARES
# ----------------------------

def ajustar_ancho_columnas(ws, dataframe, ancho_maximo=60, factor_ajuste=1.25):
    """Ajuste genérico del ancho de columnas estimando el ancho visual."""
    for i, col in enumerate(dataframe.columns, start=1):
        max_length = max((len(str(cell.value)) for cell in ws[get_column_letter(i)] if cell.value), default=0)
        ancho_estimado = min(ancho_maximo, int(max_length * factor_ajuste) + 2)
        ws.column_dimensions[get_column_letter(i)].width = max(10, ancho_estimado)

# ----------------------------
# CARGA Y LIMPIEZA DEL CSV
# ----------------------------
df = pd.read_csv("AVN-6500-01_Shelf Inventory_2025-06-20 23-40-22.csv")
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.drop(columns=[
    'Card Width', 'Mfg. Date', 'Age', 'On Since', 
    'SNMP Index', 'Current Temperature', 'Average Temperature', 'Additional Information'
])
df = df.replace('\t', '', regex=True)

# ----------------------------
# CREAR WORKBOOK
# ----------------------------
wb = openpyxl.Workbook()
wb.remove(wb.active)  # Quitar hoja vacía por defecto

# ----------------------------
# CREAR UNA HOJA POR CADA Shelf
# ----------------------------
for shelf_num, group_df in df.groupby('Shelf'):
    ws = wb.create_sheet(title=f"Shelf_{shelf_num}")

    # Insertar encabezado + datos
    for r_idx, row in enumerate(dataframe_to_rows(group_df, index=False, header=True), start=1):
        ws.append(row)

        for c_idx, _ in enumerate(row, start=1):
            cell = ws.cell(row=r_idx, column=c_idx)
            cell.alignment = center_alignment
            cell.border = thin_border

            # Encabezado (primera fila)
            if r_idx == 1:
                cell.font = header_font
                cell.fill = header_fill

    # Ajustar anchos de columnas con algoritmo genérico
    ajustar_ancho_columnas(ws, group_df, ancho_maximo=60, factor_ajuste=1.25)

    # Congelar encabezado y activar filtro automático
    ws.freeze_panes = "A2"

# ----------------------------
# GUARDAR ARCHIVO
# ----------------------------
wb.save("archivo_por_shelf.xlsx")

print("✔ Archivo generado correctamente.")


✔ Archivo generado correctamente.
